In [1]:
# define relevants paths and load functions and libraries

%run Z:\\home\\shared\\Gaia\\Coliseum\\Delays\\paper_code\\Analysis\\helper_functions\\functions_analysis.py

data_path = 'paper_code\\Datasets\\movement_control_datasets\\' # your data path
saving_path = 'paper_code\\Figures_output\\' # your saving figures path

In [2]:
# load the list of animals for this dataset

file =''.join([data_path,'movement_control_animals.txt'])
with open(file, 'r') as file:
    loaded_folder_names = [line.strip() for line in file.readlines()]
    

In [9]:
# Save average frame to plot

path = ''.join([data_path,'raw_data\\'])
videoFolders = [os.path.join(path, d) for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]

this_path = os.path.join(videoFolders[2])
sub_folder = os.listdir(this_path)

# Find the .avi video file in the folder
video_files = [f for f in sub_folder if f.endswith('.avi')]
video_path = os.path.join(this_path, video_files[0])

print(f"Loading video: {video_path}")

# Open video using cv2
cap = cv2.VideoCapture(video_path)

#%% Get video properties
Ly = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
Lx = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
nframes = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Video dimensions: {Ly}x{Lx}, Total frames: {nframes}")

# Early exit for short videos
if nframes < 200:
    print("Video is too short, skipping.")
    sys.exit(1)

# Compute avgframe and avgmotion
nf = min(2000000, nframes)
nt0 = min(2000, nframes)
nsegs = int(np.floor(nf / nt0))
tf = np.floor(np.linspace(0, nframes - nt0, nsegs)).astype(int)

avgframe = np.zeros((Ly, Lx), np.float32)
avgmotion = np.zeros((Ly, Lx), np.float32)
ns = 0

print("Calculating average frame and motion...")

for n in tqdm(range(nsegs)):
    t = tf[n]

    # Set the video position to the desired frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, t)
    frames = []

    # Read nt0 frames
    for _ in range(nt0):
        ret, frame = cap.read()
        if not ret:
            break
        # Convert to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frames.append(gray_frame)

    # Stack frames into a single array (Time x Ly x Lx)
    im = np.stack(frames, axis=0)

    # Convert to float and transpose to (Ly, Lx, Time)
    im = np.transpose(im, (1, 2, 0)).astype(np.float32)

    # Add to averages
    avgframe += im.mean(axis=-1)
    immotion = np.abs(np.diff(im, axis=-1))
    avgmotion += immotion.mean(axis=-1)
    ns += 1

avgframe /= float(ns)
avgmotion /= float(ns)

# and it get saved so it is easier for the plotting ot just load it
save_dir = ''.join([data_path,'avg_image_example.npy'])
np.save(save_dir,avgframe) 


Loading video: Z:\home\shared\Gaia\Coliseum\Delays\paper_code\Datasets\movement_control_datasets\raw_data\GB32_S2\downsample_video.avi
Video dimensions: 544x728, Total frames: 926733
Calculating average frame and motion...


100%|██████████████████████████████████████████████████████████████████████████████| 463/463 [1:42:43<00:00, 13.31s/it]


In [8]:
# Save psth of example experiment to plot in main figure 

file=os.path.join(videoFolders[2],'dec_data.mat')
data_dict = mat73.loadmat(file)
DAT=data_dict['dec_data']

Vol_motSVD=DAT['Vol_motSVD']
Vol_times=DAT['Vol_times']

num_new_times_imp = 210
resampled_motSVD = np.full((Vol_motSVD.shape[0], Vol_motSVD.shape[1], num_new_times_imp),np.nan)

for rep in range(Vol_times.shape[0]):
    
    these_times = Vol_times[rep,:]
    
    new_times = np.arange(np.min(these_times), np.max(these_times), 0.01)
    num_new_times = len(new_times)
    
    if num_new_times_imp != num_new_times:
        print('There is a mistake!')   
    for n in range(Vol_motSVD.shape[0]):
        interp_func = interp1d(these_times, Vol_motSVD[n, rep, :], kind='linear', fill_value="extrapolate")
        resampled_motSVD[n, rep, :] = interp_func(new_times) 
        
features = np.swapaxes(resampled_motSVD,0,1)
features = features.reshape(features.shape[0],-1)

# Min-Max normalization
features_min = np.min(features)
features_max = np.max(features)
features_norm = 2 * (features - features_min) / (features_max - features_min) - 1

# and it get saved so it is easier for the plotting ot just load it
save_dir = ''.join([data_path,'psth_example.npy'])
np.save(save_dir,features_norm) 


In [23]:
# for each animal load the PRED and Test and get the decoder accuracy with 1PC
# load the outputs from this code run_classifier_movement_control_AVdelays.py

these_animals = np.arange(len(loaded_folder_names))
n_animals = these_animals.shape[0]
n_rep = 20
all_scores = np.zeros((n_animals,2))

for count,animal in enumerate(these_animals):
    
    file =''.join([data_path,f'decoder_analysis\\PRED_Alldelays_Ashifted_1PC_original_animal_{animal}_SVM.npy'])     
    predicted=np.load(file)
    file =''.join([data_path,f'decoder_analysis\\TEST_Alldelays_Ashifted_1PC_original_animal_{animal}_SVM.npy'])    
    y_test=np.load(file)
    
    these_scores =  np.zeros((predicted.shape[0],1))
    for rep in range(predicted.shape[0]):
        sub_score = np.zeros((predicted.shape[1],1))
        for k in range(predicted.shape[1]):
            cm = confusion_matrix(y_test[rep,k,:], predicted[rep,k,:],normalize='true')*100
            score = np.diag(cm)
            sub_score = np.mean(score[:11])
    
        # save the scores for the main figure
        these_scores[rep] = np.mean(sub_score)
    
    # save the mean of this
    all_scores[count,0] = np.mean(these_scores)
    
    file =''.join([data_path,f'decoder_analysis\\PRED_Alldelays_Ashifted_1PC_original_random_animal_{animal}_SVM.npy'])     
    predicted=np.load(file)
    file =''.join([data_path,f'decoder_analysis\\TEST_Alldelays_Ashifted_1PC_original_random_animal_{animal}_SVM.npy'])    
    y_test=np.load(file)
    
    these_scores =  np.zeros((predicted.shape[0],1))
    for rep in range(predicted.shape[0]):
        sub_score = np.zeros((predicted.shape[1],1))
        for k in range(predicted.shape[1]):
            cm = confusion_matrix(y_test[rep,k,:], predicted[rep,k,:],normalize='true')*100
            score = np.diag(cm)
            sub_score = np.mean(score[:11])
    
        # save the scores for the main figure
        these_scores[rep] = np.mean(sub_score)
    
    # save the mean of this
    all_scores[count,1] = np.mean(these_scores)
    
# and it get saved so it is easier for the plotting ot just load it
save_dir = ''.join([data_path,'decoder_accuracy_1PC_Ashifted.npy'])
np.save(save_dir,all_scores)    


In [24]:
# for each animal load the PRED and Test and get the decoder accuracy with 100PC
# load the outputs from this code run_classifier_movement_control_AVdelays.py

these_animals = np.arange(len(loaded_folder_names))
n_animals = these_animals.shape[0]
n_rep = 20
all_scores = np.zeros((n_animals,2))

for count,animal in enumerate(these_animals):
    
    file =''.join([data_path,f'decoder_analysis\\PRED_Alldelays_Ashifted_original_animal_{animal}_SVM.npy'])     
    predicted=np.load(file)
    file =''.join([data_path,f'decoder_analysis\\TEST_Alldelays_Ashifted_original_animal_{animal}_SVM.npy'])    
    y_test=np.load(file)
    
    these_scores =  np.zeros((predicted.shape[0],1))
    for rep in range(predicted.shape[0]):
        sub_score = np.zeros((predicted.shape[1],1))
        for k in range(predicted.shape[1]):
            cm = confusion_matrix(y_test[rep,k,:], predicted[rep,k,:],normalize='true')*100
            score = np.diag(cm)
            sub_score = np.mean(score[:11])
    
        # save the scores for the main figure
        these_scores[rep] = np.mean(sub_score)
    
    # save the mean of this
    all_scores[count,0] = np.mean(these_scores)
    
    file =''.join([data_path,f'decoder_analysis\\PRED_Alldelays_Ashifted_original_random_animal_{animal}_SVM.npy'])     
    predicted=np.load(file)
    file =''.join([data_path,f'decoder_analysis\\TEST_Alldelays_Ashifted_original_random_animal_{animal}_SVM.npy'])    
    y_test=np.load(file)
    
    these_scores =  np.zeros((predicted.shape[0],1))
    for rep in range(predicted.shape[0]):
        sub_score = np.zeros((predicted.shape[1],1))
        for k in range(predicted.shape[1]):
            cm = confusion_matrix(y_test[rep,k,:], predicted[rep,k,:],normalize='true')*100
            score = np.diag(cm)
            sub_score = np.mean(score[:11])
    
        # save the scores for the main figure
        these_scores[rep] = np.mean(sub_score)
    
    # save the mean of this
    all_scores[count,1] = np.mean(these_scores)
    
# and it get saved so it is easier for the plotting ot just load it
save_dir = ''.join([data_path,'decoder_accuracy_100PC_Ashifted.npy'])
np.save(save_dir,all_scores)   


In [26]:
# for each animal load the PRED and Test and get the decoder accuracy for sound on sound off
# load the outputs from this code 

these_animals = np.arange(len(loaded_folder_names))
n_animals = these_animals.shape[0]
n_rep = 20
all_scores = np.zeros((n_animals,2))

for count,animal in enumerate(these_animals):
    file =''.join([data_path,f'decoder_analysis\\PRED_SoundNoSound_animal_{animal}_SVM.npy'])     
    predicted=np.load(file)
    file =''.join([data_path,f'decoder_analysis\\TEST_SoundNoSound_animal_{animal}_SVM.npy'])    
    y_test=np.load(file)
    
    these_scores =  np.zeros((predicted.shape[0],1))
    for rep in range(predicted.shape[0]):
        sub_score = np.zeros((predicted.shape[1],1))
        for k in range(predicted.shape[1]):
            cm = confusion_matrix(y_test[rep,k,:], predicted[rep,k,:],normalize='true')*100
            score = np.diag(cm)
            sub_score = np.mean(score)
    
        # save the scores for the main figure
        these_scores[rep] = np.mean(sub_score)
    
    # save the mean of this
    all_scores[count,0] = np.mean(these_scores)
    
    file =''.join([data_path,f'decoder_analysis\\PRED_SoundNoSound_random_animal_{animal}_SVM.npy'])     
    predicted=np.load(file)
    file =''.join([data_path,f'decoder_analysis\\TEST_SoundNoSound_random_animal_{animal}_SVM.npy'])    
    y_test=np.load(file)
    
    these_scores =  np.zeros((predicted.shape[0],1))
    for rep in range(predicted.shape[0]):
        sub_score = np.zeros((predicted.shape[1],1))
        for k in range(predicted.shape[1]):
            cm = confusion_matrix(y_test[rep,k,:], predicted[rep,k,:],normalize='true')*100
            score = np.diag(cm)
            sub_score = np.mean(score)
    
        # save the scores for the main figure
        these_scores[rep] = np.mean(sub_score)
    
    # save the mean of this
    all_scores[count,1] = np.mean(these_scores)
    
# and it get saved so it is easier for the plotting ot just load it
save_dir = ''.join([data_path,'decoder_accuracy_80dBSoundNoSound.npy'])
np.save(save_dir,all_scores)   


In [27]:
# for each animal load the PRED and Test and get the decoder accuracy for sound on sound off AT 60dB
# load the outputs from this code 

these_animals = np.arange(len(loaded_folder_names))
n_animals = these_animals.shape[0]
n_rep = 20
all_scores = np.zeros((n_animals,2))

for count,animal in enumerate(these_animals):
    file =''.join([data_path,f'decoder_analysis\\PRED_SoundNoSound60dB_animal_{animal}_SVM.npy'])     
    predicted=np.load(file)
    file =''.join([data_path,f'decoder_analysis\\TEST_SoundNoSound60dB_animal_{animal}_SVM.npy'])    
    y_test=np.load(file)
    
    these_scores =  np.zeros((predicted.shape[0],1))
    for rep in range(predicted.shape[0]):
        sub_score = np.zeros((predicted.shape[1],1))
        for k in range(predicted.shape[1]):
            cm = confusion_matrix(y_test[rep,k,:], predicted[rep,k,:],normalize='true')*100
            score = np.diag(cm)
            sub_score = np.mean(score)
    
        # save the scores for the main figure
        these_scores[rep] = np.mean(sub_score)
    
    # save the mean of this
    all_scores[count,0] = np.mean(these_scores)
    
    file =''.join([data_path,f'decoder_analysis\\PRED_SoundNoSound60dB_random_animal_{animal}_SVM.npy'])     
    predicted=np.load(file)
    file =''.join([data_path,f'decoder_analysis\\TEST_SoundNoSound60dB_random_animal_{animal}_SVM.npy'])    
    y_test=np.load(file)
    
    these_scores =  np.zeros((predicted.shape[0],1))
    for rep in range(predicted.shape[0]):
        sub_score = np.zeros((predicted.shape[1],1))
        for k in range(predicted.shape[1]):
            cm = confusion_matrix(y_test[rep,k,:], predicted[rep,k,:],normalize='true')*100
            score = np.diag(cm)
            sub_score = np.mean(score)
    
        # save the scores for the main figure
        these_scores[rep] = np.mean(sub_score)
    
    # save the mean of this
    all_scores[count,1] = np.mean(these_scores)
    
# and it get saved so it is easier for the plotting ot just load it
save_dir = ''.join([data_path,'decoder_accuracy_60dBSoundNoSound.npy'])
np.save(save_dir,all_scores)   


In [28]:
# for each animal load the PRED and Test and get the decoder accuracy for vis stim on and off
# load the outputs from this code 

n_rep = 20
all_scores = np.zeros((n_animals,2))

for count,animal in enumerate(these_animals):
    file =''.join([data_path,f'decoder_analysis\\PRED_StimNoStim_Vis_animal_{animal}_SVM.npy'])     
    predicted=np.load(file)
    file =''.join([data_path,f'decoder_analysis\\TEST_StimNoStim_Vis_animal_{animal}_SVM.npy'])    
    y_test=np.load(file)
    
    these_scores =  np.zeros((predicted.shape[0],1))
    for rep in range(predicted.shape[0]):
        sub_score = np.zeros((predicted.shape[1],1))
        for k in range(predicted.shape[1]):
            cm = confusion_matrix(y_test[rep,k,:], predicted[rep,k,:],normalize='true')*100
            score = np.diag(cm)
            sub_score = np.mean(score)
    
        # save the scores for the main figure
        these_scores[rep] = np.mean(sub_score)
    
    # save the mean of this
    all_scores[count,0] = np.mean(these_scores)
    
    file =''.join([data_path,f'decoder_analysis\\PRED_StimNoStim_Vis_random_animal_{animal}_SVM.npy'])     
    predicted=np.load(file)
    file =''.join([data_path,f'decoder_analysis\\TEST_StimNoStim_Vis_random_animal_{animal}_SVM.npy'])    
    y_test=np.load(file)

    these_scores =  np.zeros((predicted.shape[0],1))
    for rep in range(predicted.shape[0]):
        sub_score = np.zeros((predicted.shape[1],1))
        for k in range(predicted.shape[1]):
            cm = confusion_matrix(y_test[rep,k,:], predicted[rep,k,:],normalize='true')*100
            score = np.diag(cm)
            sub_score = np.mean(score)
    
        # save the scores for the main figure
        these_scores[rep] = np.mean(sub_score)
    
    # save the mean of this
    all_scores[count,1] = np.mean(these_scores)

# and it get saved so it is easier for the plotting ot just load it
save_dir = ''.join([data_path,'decoder_accuracy_StimNoStim_Vis.npy'])
np.save(save_dir,all_scores)   


In [45]:
# extract average PC1 for different sound intensities

these_animals = np.arange(len(loaded_folder_names))
n_vol = 9
avg_pc1 = np.zeros((these_animals.shape[0],n_vol))

this_path = ''.join([data_path,'raw_data\\'])
videoFolders = [os.path.join(this_path, d) for d in os.listdir(this_path) if os.path.isdir(os.path.join(this_path, d))]

for count1,animal in tqdm(enumerate(range(these_animals.shape[0]))):
    file=os.path.join(videoFolders[animal],'dec_data.mat')
    data_dict = mat73.loadmat(file)
    DAT=data_dict['dec_data']

    Del_motSVD=DAT['Vol_motSVD']
    Del_times=DAT['Vol_times']
    Del_trials=DAT['Vol_trials']

    # 10 ms resolution seem to be working well
    num_new_times_imp = 210#420
    resampled_motSVD = np.full((Del_motSVD.shape[0], Del_motSVD.shape[1], num_new_times_imp),np.nan)

    for rep in range(Del_times.shape[0]):

        these_times = Del_times[rep,:]
        
        # 1. Define new time grid with 10ms (0.01s) intervals
        new_times = np.arange(np.min(these_times), np.max(these_times), 0.01)
        num_new_times = len(new_times)

        if num_new_times_imp != num_new_times:
            print('There is a mistake!')   
        for n in range(Del_motSVD.shape[0]):
            interp_func = interp1d(these_times, Del_motSVD[n, rep, :], kind='linear', fill_value="extrapolate")
            resampled_motSVD[n, rep, :] = interp_func(new_times)
            
    sub_resampled_motSVD = resampled_motSVD[0,:,40:-60]
    max_vol = np.mean(sub_resampled_motSVD[-50:,:],axis=0)
    abs_max = max_vol[np.argmax(np.abs(max_vol))]
    
    if abs_max <0:
        # make sure the 1st PC is always positive
        sub_resampled_motSVD = sub_resampled_motSVD*-1   
    
    original_shape = sub_resampled_motSVD.shape  # (700, 110)

    # 1. Reshape to 1D
    flattened = sub_resampled_motSVD.ravel()  # or .reshape(-1)
    # 2. Z-score normalization
    z_scored = zscore(flattened)

    # 3. Reshape back to original shape
    z_scored_reshaped = z_scored.reshape(original_shape)

    mean_sub_resampled_motSVD = np.mean(z_scored_reshaped[:,10:60],axis=1)

    n_rep = 50
    my_labels = np.repeat(np.arange(mean_sub_resampled_motSVD.shape[0]/n_rep),n_rep)

    for count2,i in enumerate(np.unique(my_labels)):
        avg_pc1[count1,count2] = np.mean(mean_sub_resampled_motSVD[my_labels==i])

# and it get saved so it is easier for the plotting ot just load it
save_dir = ''.join([data_path,'z_scored_PC1_sound_intensity.npy'])
np.save(save_dir,avg_pc1)   


9it [01:27,  9.76s/it]


In [46]:
# extract average PC1 for different AV delays

these_animals = np.arange(len(loaded_folder_names))
n_stim = 14
avg_pc1 = np.zeros((these_animals.shape[0],n_stim))

for count1,animal in tqdm(enumerate(range(9))):
    file=os.path.join(videoFolders[animal],'dec_data.mat')

    data_dict = mat73.loadmat(file)
    DAT=data_dict['dec_data']

    Del_motSVD=DAT['Del_motSVD']
    Del_times=DAT['Del_times']
    Del_trials=DAT['Del_trials']

    # 10 ms resolution seem to be working well
    num_new_times_imp = 210#420
    resampled_motSVD = np.full((Del_motSVD.shape[0], Del_motSVD.shape[1], num_new_times_imp),np.nan)

    for rep in range(Del_times.shape[0]):

        these_times = Del_times[rep,:]
        # 1. Define new time grid with 10ms (0.01s) intervals
        new_times = np.arange(np.min(these_times), np.max(these_times), 0.01)
        num_new_times = len(new_times)

        if num_new_times_imp != num_new_times:
            print('There is a mistake!')   
        for n in range(Del_motSVD.shape[0]):
            interp_func = interp1d(these_times, Del_motSVD[n, rep, :], kind='linear', fill_value="extrapolate")
            resampled_motSVD[n, rep, :] = interp_func(new_times) 

    sub_resampled_motSVD = resampled_motSVD[0,:,40:-60]
    
    max_vol = np.mean(sub_resampled_motSVD[600:650,:],axis=0)    
    abs_max = max_vol[np.argmax(np.abs(max_vol))]
    
    if abs_max <0:
        sub_resampled_motSVD = sub_resampled_motSVD*-1
    
    original_shape = sub_resampled_motSVD.shape  # (700, 110)

    # 1. Reshape to 1D
    flattened = sub_resampled_motSVD.ravel()  # or .reshape(-1)
    # 2. Z-score normalization
    z_scored = zscore(flattened)

    # 3. Reshape back to original shape
    z_scored_reshaped = z_scored.reshape(original_shape)
    
    mean_sub_resampled_motSVD = np.mean(z_scored_reshaped[:,10:80],axis=1)
    
    n_rep = 50
    my_labels = np.repeat(np.arange(mean_sub_resampled_motSVD.shape[0]/n_rep),n_rep)

    for count2,i in enumerate(np.unique(my_labels)):
        avg_pc1[count1,count2] = np.mean(mean_sub_resampled_motSVD[my_labels==i])
        
# and it get saved so it is easier for the plotting ot just load it
save_dir = ''.join([data_path,'z_scored_PC1_AVdelays.npy'])
np.save(save_dir,avg_pc1)   

9it [01:47, 11.90s/it]
